In [123]:
from azure.identity import ClientSecretCredential
from azure.storage.blob import BlobServiceClient
# from dotenv import load_dotenv
import os

import pandas as pd
from pandas import ExcelWriter

In [ ]:
load_dotenv()

In [ ]:
client_id = os.environ['AZURE_CLIENT_ID']
tenant_id = os.environ['AZURE_TENANT_ID']
client_secret = os.environ['AZURE_CLIENT_SECRET']
account_url = os.environ["AZURE_STORAGE_URL"]

In [ ]:
# create a credential
credentials = ClientSecretCredential(
    client_id = client_id,
    client_secret = client_secret,
    tenant_id = tenant_id
)

In [124]:
df_xm = pd.read_csv('./input/reporte_XM.csv', sep='|')
df_xm.head()

,IdExtraccion,Tema,NombreExtraccion,IdExtraccionMaestra,IndRegulatorio,DatoObligatorio,PLDataFactory,ValorDeltaInicial,ValorDeltaFinal,tipo_liqui,estado_deltas,EstadoPipeline,Descripcion,Ejecucion,FechaInicioEjecucion,Estado
0,354,InformacionTransaccionesMEM,Multifuente_17_196649_u,624.0,False,False,PL_SIMEM_Maestras_Liquidacion_Diaria,2024-08-28,2024-08-30,Liquidacion,Deltas OK!,Finalizado,NaN,NaN,NaN,Finalizado
1,380,InformacionTransaccionesMEM,Multifuente_17_196649_x,624.0,False,False,PL_SIMEM_Maestras_Liquidacion_Diaria,2024-08-28,2024-08-30,Liquidacion,Deltas OK!,Finalizado,NaN,NaN,NaN,Finalizado
2,389,InformacionTransaccionesMEM,Multifuente_17_196649_k,624.0,False,False,PL_SIMEM_Maestras_Liquidacion_Diaria,2024-08-28,2024-08-30,Liquidacion,Deltas OK!,Finalizado,NaN,NaN,NaN,Finalizado
3,395,InformacionTransaccionesMEM,Multifuente_17_196649_o,624.0,False,False,PL_SIMEM_Maestras_Liquidacion_Diaria,2024-08-28,2024-08-30,Liquidacion,Deltas OK!,Finalizado,NaN,NaN,NaN,Finalizado
4,400,InformacionTransaccionesMEM,Multifuente_17_196649_zc,624.0,False,False,PL_SIMEM_Maestras_Liquidacion_Diaria,2024-08-28,2024-08-30,Liquidacion,Deltas OK!,Finalizado,NaN,NaN,NaN,Finalizado


In [125]:
df_xm = df_xm.loc[((df_xm['EstadoPipeline'].isin(['Error', 'Iniciado'])) & (df_xm['Tema'] != 'Metricas')) |
           ((df_xm['EstadoPipeline'] == 'Finalizado') & (df_xm['estado_deltas'] != 'Deltas OK!') & (df_xm['Tema'] != 'Metricas'))]

In [126]:
df_xm = df_xm.loc[:, ['IdExtraccion',  'NombreExtraccion', 'Estado', 'estado_deltas', 'EstadoPipeline', 'Descripcion']]

In [127]:
df_xm

,IdExtraccion,NombreExtraccion,Estado,estado_deltas,EstadoPipeline,Descripcion
46,320,10_ResumenInformacionOperativaGeneracion,Fallido,"Problemas con deltas, se tienen 2024-09-01 y 2...",Error,Lectura dataframe - Error de valor - No se enc...
94,637,Liquidacion_Mensual,Fallido,"Problemas con deltas, se tienen 2024-08-31 y 2...",Error,Pipeline - Fallo el notebook de curated al eje...
154,898,Multifuente_38_ReporteContratosLP,Fallido,"Problemas con deltas, se tienen 2024-06-01 y 2...",Iniciado,NaN


In [128]:
df_simem = pd.read_csv('./input/reporte_SIMEM.csv', sep='|')

In [129]:
df_simem = df_simem.loc[((df_simem['EstadoPipeline'].isin(['Error', 'Iniciado'])) & (df_simem['Tema'] != 'Metricas')) |
           ((df_simem['EstadoPipeline'] == 'Finalizado') & (df_simem['estado_deltas'] != 'Deltas OK!') & (df_simem['Tema'] != 'Metricas'))]
df_simem.fillna({'Descripcion': ''}, inplace=True)
df_simem = df_simem.loc[(~(df_simem['Descripcion'].str.startswith('Guardado - Error de valor - No se pudo crear')))]

In [130]:
df_simem = df_simem.loc[:, ['IdExtraccion', 'IdConfiguracionGeneracionArchivos', 'NombreArchivoDestino', 'Estado', 'estado_deltas', 'EstadoPipeline', 'Descripcion']]

In [131]:
df_simem

,IdExtraccion,IdConfiguracionGeneracionArchivos,NombreArchivoDestino,Estado,estado_deltas,EstadoPipeline,Descripcion
4,737,0837F055-DDDA-4100-B546-CCFD7F879F26,ConsumoCombustibleMedianoPlazo,Fallido,"Problemas con deltas, se tienen 2024-08-26 y 2...",Error,Pipeline - Fallo ejecucion del NoteBook (Error...
9,730,1088A60B-BDFB-47CC-9E78-E49B68DC34F9,ImportacionesNetasEnergeticoMedianoPlazo,Fallido,"Problemas con deltas, se tienen 2024-08-26 y 2...",Error,Pipeline - Fallo ejecucion del NoteBook (Error...
12,658,135C1099-3F6A-4978-938C-4DDD4C3571BF,ValoresCargoPorConfiabilidad,Fallido,Falta configurar alerta de delta,Finalizado,
13,629,135C1099-3F6A-4978-938C-4DDD4C3571BF,ValoresCargoPorConfiabilidad,Fallido,Falta configurar alerta de delta,Finalizado,
24,593,1C91C4A6-718A-42F2-8642-D211D4EB515B,ListadoRetiroUnidades,Fallido,"Problemas con deltas, se tienen 2024-09-02 y 2...",Finalizado,
25,599,1C91C4A6-718A-42F2-8642-D211D4EB515B,ListadoRetiroUnidades,Fallido,"Problemas con deltas, se tienen 2024-09-02 y 2...",Finalizado,
30,728,1EEE6372-EE93-4DAA-AA1E-4083911A0ABC,AportesEnergeticoMedianoPlazo,Fallido,"Problemas con deltas, se tienen 2024-08-26 y 2...",Error,Pipeline - Fallo ejecucion del NoteBook (Error...
40,375,344FC94D-B86A-4266-BF7B-246F73357220,ParametrosTecnicosPlantaUnidadEolica,Fallido,Deltas OK!,Error,Pipeline - Fallo ejecucion del NoteBook (Error...
42,806,37A5084A-48D9-474A-8C5F-1C7786639796,10_PanoramaEnergeticoCortoPlazoEmbalse,Fallido,"Problemas con deltas, se tienen 2024-09-02 y 2...",Error,Pipeline - Fallo ejecucion del NoteBook (Error...
55,525,4F20758E-2E8F-421D-986C-3250849614CC,MantenimientosGeneradores,Fallido,"Problemas con deltas, se tienen 2024-09-02 y 2...",Error,Pipeline - Fallo ejecucion del NoteBook (Error...


In [136]:
df_regulatorios = pd.read_excel('./input/Regulatorios.xlsx')
# df_regulatorios = df_regulatorios.loc[(df_regulatorios['DiaRevision'].isin(['SABADO'])) & (df_regulatorios['Hora'] >  17)]
df_regulatorios = df_regulatorios.loc[(df_regulatorios['DiaRevision'].isin(['DOMINGO'])) & (df_regulatorios['Hora'] > 18)]
df_regulatorios


,PlazoMaximoReporte,FechaReporte,FechaInformacion,DiaRevision,NombreArchivoDestino,IdConfiguracionGeneracionArchivos,CodigoDelta,Periodicidad,Hora
9,2024-09-02 00:00:00.000,NaN,2024-08-30 00:00:00.000,DOMINGO,CondiciondelSistemaEstatutoDesabastecimientoSe...,EBC2D18A-51D8-4122-8E48-1E80E57153A0,DP,Diaria,19
14,2024-09-02 00:00:00.000,NaN,2024-08-30 00:00:00.000,DOMINGO,PrecioMarginalesDiarioRedespacho,03E35F17-E237-4C84-A51E-173D72942EB2,DP,Diaria,23
24,2024-09-02 00:00:00.000,NaN,2024-09-01 00:00:00.000,DOMINGO,InformeDespacho,D0B4E32D-4858-4791-898E-E88B929C550F,DF,Diaria,20


In [137]:
df_regulatorios.merge(df_simem, on='IdConfiguracionGeneracionArchivos', how='inner')

,PlazoMaximoReporte,FechaReporte,FechaInformacion,DiaRevision,NombreArchivoDestino_x,IdConfiguracionGeneracionArchivos,CodigoDelta,Periodicidad,Hora,IdExtraccion,NombreArchivoDestino_y,Estado,estado_deltas,EstadoPipeline,Descripcion


In [122]:
with ExcelWriter('./output/01092024_Conjuntos.xlsx') as writer:
    df_simem.to_excel(writer, 'SIMEM')
    df_xm.to_excel(writer, 'XM')

/tmp/ipykernel_58440/549430417.py:2: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  df_simem.to_excel(writer, 'SIMEM')
/tmp/ipykernel_58440/549430417.py:3: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  df_xm.to_excel(writer, 'XM')
